In [1]:
%run base.ipynb

jit after {'jit': True, 'jit_options': {'flags': '-Ofast'}, 'compiler': 'shell'}
number of joints = 4


In [2]:
# forward dynamics
x_next, rhs_xdd, states, u, dt, q_min, q_max ,arm_p, _ , _, _  = rig_dyn.forward_dynamics(gravity=9.81)

# forces and moments at the base
F_at_base = rig_dyn.get_base_forces(gravity=9.81)
F_at_base_eval = cs.Function(
    'F_at_base', [arm_ss.q, arm_ss.q_dot, arm_ss.q_ddot, arm_ss.sim_p, arm_ss.base_T], [F_at_base])

F_minus_xdd = F_at_base_eval(arm_ss.q, arm_ss.q_dot, rhs_xdd, arm_ss.sim_p, arm_ss.base_T)

In [3]:
F_next_ = cs.Function('Mnext', [states, u, arm_p, dt, q_min, q_max, arm_ss.base_T], [x_next, F_minus_xdd])
# F_next_.save("ARM_model.casadi")

In [4]:
q0 = [1.91318, 1.33339, 0.99245, 3.16073]
q0dot = [0.00751799, -0.0073626, 0.00281749, 0.0222385]
u0 = [0.0340454, -0.00441695, 0.0062739, -0.00505981]
state0 = cs.vertcat(q0, q0dot)

nextState , f_eval_base  = F_next_(state0, u0, alpha.sim_p, alpha.delta_t, alpha.joint_min, alpha.joint_max,alpha.base_T0)
print(nextState)

[1.91365, 1.3333, 0.992578, 3.16057, 0.0136182, -0.00232471, 0.00369053, -0.00505981]


In [6]:
# c , cpp or matlab code generation for forward dynamics
F_next_.generate("F_next_.c")
os.system(f"gcc -fPIC -shared F_next_.c -o libMnext.so")

0